In [4]:
# Tem que dar pip install nisso pra ele baixar só os texto em português

!pip install langdetect

UsageError: Line magic function `%%!pip` not found.


In [5]:
# Importa as bibliotecas
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from langdetect import detect

In [ ]:
# Lista de autores (Fernando Pessoa + 3 heterônimos)
autores_desejados = {"Alberto Caeiro", "Álvaro de Campos", "Ricardo Reis", "Fernando Pessoa"}

# Lista para armazenar os dados filtrados
data = []

# Loop pelas páginas de 1 a 4544 (início-fim)
for i in range(1, 4580): # Botei umas a mais
    url = f"http://arquivopessoa.net/textos/{i}"
    response = requests.get(url)

    # Tem que ter isso pra ele vir com os acento tudo certinho
    response.encoding = 'utf-8'

    # Verifica se a página existe
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        # Encontra o autor do texto (div class="autor")
        autor_tag = soup.find('div', class_='autor')
        autor = autor_tag.text.strip() if autor_tag else "Autor desconhecido"

        if autor in autores_desejados:
            # Encontra o título do texto (tag h1 com class="titulo-texto")
            titulo_tag = soup.find('h1', class_='titulo-texto')
            titulo = titulo_tag.text.strip() if titulo_tag else f'Título não encontrado ({i})'

            # Encontra o conteúdo do texto (div com class "texto-prosa" ou "texto-poesia")
            conteudo_tag = soup.find('div', class_='texto-prosa') or soup.find('div', class_='texto-poesia')
            conteudo = conteudo_tag.text.strip() if conteudo_tag else "Conteúdo não encontrado"

            # Tem que ter isso pra ele vir com os acento tudo certinho
            conteudo = conteudo.encode('utf-8').decode('utf-8')

            # Verifica o tipo de texto (prosa ou poesia)
            tipo_texto = "texto-prosa" if soup.find('div', class_='texto-prosa') else "texto-poesia" if soup.find('div', class_='texto-poesia') else "unknown"

            # Encontra referência (div class="biblio")
            biblio_tag = soup.find('div', class_='biblio')
            ref_bib = biblio_tag.text.strip() if biblio_tag else "Referência não encontrada"

            ############# IDIOMA #############
            # Detecta o idioma do conteúdo
            try:
                idioma = detect(conteudo)  # Detecta o idioma
            except:
                idioma = 'unknown'

            # Filtra apenas textos em português
            if idioma == 'pt':
                # Adiciona os dados à lista
                data.append({'ID': i, 'Título': titulo, 'Autor': autor, 'Tipo': tipo_texto, 'Texto': conteudo, 'Ref Bib': ref_bib})
                print(f"Coletado: {titulo} - {autor} (ID {i} - {idioma} - type: {tipo_texto})")  # Log para ver se deu certo
            else:
                print(f"Texto ignorado (Página {i}) - Idioma: {idioma} - type: {tipo_texto}")  # Log para textos não em português
            ##################################
    else:
        print(f"Erro na página {i}, Status Code: {response.status_code}") # Caso dê erro na página (erro 404: página não existe, erro 503: erro de requisição (tá muito rápido))

    # Tem que ter isso se não dá pau no site
    time.sleep(1)

# Cria o DataFrame e salva em CSV
df = pd.DataFrame(data)
df.to_csv('arquivopessoa_textos_filtrados.csv', index=False, encoding='utf-8-sig')

print("Filé papai! Dados salvos em 'arquivopessoa_textos_filtrados.csv'")

Erro na página 1, Status Code: 404
Erro na página 2, Status Code: 404
Erro na página 3, Status Code: 404
Texto ignorado (Página 4) - Idioma: es - type: texto-poesia
Coletado: A REFORMA DO CALENDÁRIO E AS SUAS CONSEQUÊNCIAS COMERCIAIS - Fernando Pessoa (ID 5 - pt - type: texto-prosa)
Texto ignorado (Página 6) - Idioma: en - type: texto-prosa
Coletado: Aqui, sem outro Apolo do que Apolo, - Ricardo Reis (ID 7 - pt - type: texto-poesia)
Texto ignorado (Página 8) - Idioma: en - type: texto-prosa
Coletado: Em Ceres anoitece. - Ricardo Reis (ID 9 - pt - type: texto-poesia)
Texto ignorado (Página 10) - Idioma: en - type: texto-prosa
Texto ignorado (Página 11) - Idioma: en - type: texto-prosa
Coletado: Não a ti, mas aos teus, odeio, Cristo. - Ricardo Reis (ID 12 - pt - type: texto-poesia)
Coletado: O burburinho da água - Fernando Pessoa (ID 13 - pt - type: texto-poesia)
Coletado: O dualismo é basilar na ciência; - Fernando Pessoa (ID 15 - pt - type: texto-prosa)
Coletado: Ou o infinito se pode 

In [ ]:
# Lê o CSV
csv_path = "arquivopessoa_textos_filtrados.csv"
df_1 = pd.read_csv(csv_path)

# Mostra o DataFrame
df_1

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,ID,Título,Autor,Tipo,Texto,Ref Bib
0,4,Diana através dos ramos,Ricardo Reis,texto-poesia,Diana através dos ramos\nEspreita a vinda de E...,Poemas de Ricardo Reis.\n\n Fernando Pessoa. (...
1,5,A REFORMA DO CALENDÁRIO E AS SUAS CONSEQUÊNCIA...,Fernando Pessoa,texto-prosa,A REFORMA DO CALENDÁRIO E AS SUAS CONSEQUÊNCIA...,Páginas de Pensamento Político. Vol II. Fernan...
2,7,"Aqui, sem outro Apolo do que Apolo,",Ricardo Reis,texto-poesia,"Aqui, sem outro Apolo do que Apolo,\nSem um su...",Poemas de Ricardo Reis.\n\n Fernando Pessoa. (...
3,9,Em Ceres anoitece.,Ricardo Reis,texto-poesia,Em Ceres anoitece.\nNos píncaros ainda\n ...,Poemas de Ricardo Reis.\n\n Fernando Pessoa. (...
4,12,"Não a ti, mas aos teus, odeio, Cristo.",Ricardo Reis,texto-poesia,"Não a ti, mas aos teus, odeio, Cristo.\nTu não...",Poemas de Ricardo Reis.\n\n Fernando Pessoa. (...
...,...,...,...,...,...,...
3137,4539,Bate a luz no cimo,Fernando Pessoa,texto-prosa,"Bate a luz no cimo Da montanha, vê... Sem quer...",Cartas de Fernando Pessoa a Armando Côrtes-Rod...
3138,4540,Vai redonda e alta,Fernando Pessoa,texto-poesia,Vai redonda e alta A lua. Que dor É em mim um ...,Cartas de Fernando Pessoa a Armando Côrtes-Rod...
3139,4541,Saber? Que sei eu?,Fernando Pessoa,texto-poesia,Saber? Que sei eu? Pensar é descrer. — Leve ...,Cartas de Fernando Pessoa a Armando Côrtes-Rod...
3140,4542,Sopra de mais o vento,Fernando Pessoa,texto-poesia,Sopra de mais o vento Para eu poder descansar....,Cartas de Fernando Pessoa a Armando Côrtes-Rod...


In [ ]:
# Conta quantidade
count_tipo = df_1['Tipo'].value_counts()
count_autor = df_1['Autor'].value_counts()

# Conta quantos textos são 'texto-poesia' e 'texto-prosa' por autor
count_autor_tipo = pd.crosstab(df_1['Autor'], df_1['Tipo'])

print(count_tipo)
print()
print(count_autor)
print()
print(count_autor_tipo)

Tipo
texto-poesia    1905
texto-prosa     1237
Name: count, dtype: int64

Autor
Fernando Pessoa     2315
Álvaro de Campos     393
Ricardo Reis         307
Alberto Caeiro       127
Name: count, dtype: int64

Tipo              texto-poesia  texto-prosa
Autor                                      
Alberto Caeiro             120            7
Fernando Pessoa           1210         1105
Ricardo Reis               253           54
Álvaro de Campos           322           71


In [ ]:
# Carrega o .csv dnv (não precisava mas blz)
df_2 = pd.read_csv("arquivopessoa_textos_filtrados.csv", encoding="utf-8-sig")

# Substitui '\n' (quebra de linha) por espaço e coloca todas as palavras em caixa baixa em todas as colunas de texto
df_2 = df_2.applymap(lambda x: x.replace("\n", " ").lower() if isinstance(x, str) else x)

# Salva em um novo .csv
df.to_csv("arquivopessoa_textos_filtrados_limpo.csv", index=False, encoding="utf-8-sig")

print("Arquivo filé papai, sem quebras de linha e tudo em caixa baixa!")

Arquivo filé papai, sem quebras de linha e tudo em caixa baixa!


<ipython-input-36-df2758ac78fe>:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_2 = df_2.applymap(lambda x: x.replace("\n", " ").lower() if isinstance(x, str) else x)


In [ ]:
df_2

,ID,Título,Autor,Tipo,Texto,Ref Bib
0,4,diana através dos ramos,ricardo reis,texto-poesia,diana através dos ramos espreita a vinda de en...,poemas de ricardo reis. fernando pessoa. (ed...
1,5,a reforma do calendário e as suas consequência...,fernando pessoa,texto-prosa,a reforma do calendário e as suas consequência...,páginas de pensamento político. vol ii. fernan...
2,7,"aqui, sem outro apolo do que apolo,",ricardo reis,texto-poesia,"aqui, sem outro apolo do que apolo, sem um sus...",poemas de ricardo reis. fernando pessoa. (ed...
3,9,em ceres anoitece.,ricardo reis,texto-poesia,em ceres anoitece. nos píncaros ainda ...,poemas de ricardo reis. fernando pessoa. (ed...
4,12,"não a ti, mas aos teus, odeio, cristo.",ricardo reis,texto-poesia,"não a ti, mas aos teus, odeio, cristo. tu não ...",poemas de ricardo reis. fernando pessoa. (ed...
...,...,...,...,...,...,...
3137,4539,bate a luz no cimo,fernando pessoa,texto-prosa,"bate a luz no cimo da montanha, vê... sem quer...",cartas de fernando pessoa a armando côrtes-rod...
3138,4540,vai redonda e alta,fernando pessoa,texto-poesia,vai redonda e alta a lua. que dor é em mim um ...,cartas de fernando pessoa a armando côrtes-rod...
3139,4541,saber? que sei eu?,fernando pessoa,texto-poesia,saber? que sei eu? pensar é descrer. — leve ...,cartas de fernando pessoa a armando côrtes-rod...
3140,4542,sopra de mais o vento,fernando pessoa,texto-poesia,sopra de mais o vento para eu poder descansar....,cartas de fernando pessoa a armando côrtes-rod...


In [ ]:
# Carrega o .csv dnv (não precisava mas blz)
df_3 = pd.read_csv("arquivopessoa_textos_filtrados_limpo.csv", encoding="utf-8-sig")

# Remove os textos do tipo "texto-prosa"
df_3 = df[df["Tipo"] == "texto-poesia"]

# Salva em um novo .csv
df.to_csv("arquivopessoa_textos_poemas_limpo_poesia.csv", index=False, encoding="utf-8-sig")

print("Filé papai, .csv só com poesia!")

Filé papai, .csv só com poesia!


In [ ]:
df_3

,ID,Título,Autor,Tipo,Texto,Ref Bib
0,4,diana através dos ramos,ricardo reis,texto-poesia,diana através dos ramos espreita a vinda de en...,poemas de ricardo reis. fernando pessoa. (ed...
2,7,"aqui, sem outro apolo do que apolo,",ricardo reis,texto-poesia,"aqui, sem outro apolo do que apolo, sem um sus...",poemas de ricardo reis. fernando pessoa. (ed...
3,9,em ceres anoitece.,ricardo reis,texto-poesia,em ceres anoitece. nos píncaros ainda ...,poemas de ricardo reis. fernando pessoa. (ed...
4,12,"não a ti, mas aos teus, odeio, cristo.",ricardo reis,texto-poesia,"não a ti, mas aos teus, odeio, cristo. tu não ...",poemas de ricardo reis. fernando pessoa. (ed...
5,13,o burburinho da água,fernando pessoa,texto-poesia,o burburinho da água o burburinho da água no ...,quadras ao gosto popular. fernando pesso...
...,...,...,...,...,...,...
3136,4538,como a noite é longa!,fernando pessoa,texto-poesia,como a noite é longa! toda a noite é assim... ...,cartas de fernando pessoa a armando côrtes-rod...
3138,4540,vai redonda e alta,fernando pessoa,texto-poesia,vai redonda e alta a lua. que dor é em mim um ...,cartas de fernando pessoa a armando côrtes-rod...
3139,4541,saber? que sei eu?,fernando pessoa,texto-poesia,saber? que sei eu? pensar é descrer. — leve ...,cartas de fernando pessoa a armando côrtes-rod...
3140,4542,sopra de mais o vento,fernando pessoa,texto-poesia,sopra de mais o vento para eu poder descansar....,cartas de fernando pessoa a armando côrtes-rod...


In [ ]:
# Conta quantidade
count_tipo = df_3['Tipo'].value_counts()
count_autor = df_3['Autor'].value_counts()

# Conta quantos textos são 'texto-poesia' e 'texto-prosa' por autor (texto-prosa não vai aparecer pq foi removido)
count_autor_tipo = pd.crosstab(df['Autor'], df['Tipo'])

print(count_tipo)
print()
print(count_autor)
print()
print(count_autor_tipo)

Tipo
texto-poesia    1905
Name: count, dtype: int64

Autor
fernando pessoa     1210
álvaro de campos     322
ricardo reis         253
alberto caeiro       120
Name: count, dtype: int64

Tipo              texto-poesia
Autor                         
alberto caeiro             120
fernando pessoa           1210
ricardo reis               253
álvaro de campos           322


In [ ]:
# Carrega dnv
df = pd.read_csv("/content/drive/My Drive/trabalho_PLN/textos/arquivopessoa_textos_poemas_limpo_poesia.csv")

# Filtrar os textos para cada autor
df_fernando_pessoa = df[df['Autor'] == 'fernando pessoa']
df_alberto_caeiro = df[df['Autor'] == 'alberto caeiro']
df_ricardo_reis = df[df['Autor'] == 'ricardo reis']
df_alvaro_de_campos = df[df['Autor'] == 'álvaro de campos']

# Salvar os DataFrames filtrados em novos arquivos CSV
df_fernando_pessoa.to_csv("fernando_pessoa.csv", index=False, encoding="utf-8-sig")
df_alberto_caeiro.to_csv("alberto_caeiro.csv", index=False, encoding="utf-8-sig")
df_ricardo_reis.to_csv("ricardo_reis.csv", index=False, encoding="utf-8-sig")
df_alvaro_de_campos.to_csv("alvaro_de_campos.csv", index=False, encoding="utf-8-sig")

print("Filé papai!")

Filé papai!
